<a href="https://colab.research.google.com/github/Kushal190903/excessive-absenteeism-detection/blob/main/unreliable_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchain_huggingface
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.2 MB/s eta 0:00:00


In [41]:
import pandas as pd
import re
import string
import numpy as np
# from time import perf_counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
# from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Dropout
from keras.regularizers import l2
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics import classification_report,confusion_matrix
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [4]:
dataframe1 = pd.read_csv("/content/drive/MyDrive/fake news detection files/train.csv")
dataframe1=dataframe1[['text','label']]
dataframe2_fake = pd.read_csv("/content/drive/MyDrive/fake news detection files/faken.csv")
dataframe2_true = pd.read_csv("/content/drive/MyDrive/fake news detection files/true.csv")
dataframe2_true["label"] = 0
dataframe2_fake["label"] = 1
# print(dataframe2_true.head(3))
# print(dataframe2_fake.head(3))
dataframe2_merge = pd.concat([dataframe2_fake, dataframe2_true], axis=0)

dataframe2 = dataframe2_merge.drop(["title", "subject","date"], axis=1)

# dataframe2.drop(["index"], axis=1, inplace=True)
dataframe=pd.concat([dataframe1,dataframe2],axis=0)
dataframe=dataframe.sample(frac=1)
dataframe.reset_index(inplace=True)
dataframe.drop(["index"], axis=1, inplace=True)
dataframe

,text,label
0,"After they were crushed in the 2012 election, ...",1
1,Fox News host Kimberly Guilfoyle has suggested...,0
2,BUENOS AIRES (Reuters) - Argentine President M...,0
3,Trump supporter gets revenge on lawn-sign vand...,1
4,AUDIO OF KARL ROVE ON HUGH HEWITT Picking ...,1
...,...,...
65714,.@SpeakerRyan the women of America are watchin...,1
65715,Liberal Tom “Rhymes With Liar” Steyer has be...,0
65716,"The driver pushed his way through the crowd, w...",1
65717,Students at a North Carolina high school have ...,1


In [5]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65719 entries, 0 to 65718
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    65680 non-null  object
 1   label   65719 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [64]:
print(dataframe.iloc[1]['text'])

Fox News host Kimberly Guilfoyle has suggested she would be interested in replacing Sean Spicer as White House press secretary, amid reports Spicer could soon be released from the role. [In an interview with Mercury News, Guilfoyle said: “I’m a patriot and it would be an honor to serve the country. I think it’d be a fascinating job. It’s a challenging job, and you need someone really determined and focused, a great communicator in there with deep knowledge to be able to handle that position. ”  Earlier this week, The New York Times reported that President Trump was considering Guilfoyle as a replacement for Spicer, whose time as press secretary has been marked by a number of gaffes and mistakes. An anonymously sourced report by ABC News also alleges that Trump is furious with his communications team over recent media narratives and could fire both Spicer and his communications director Mike Dubke. Guilfoyle, 48, is the former first lady of San Fransisco, having been married to former c

##wordopt can be used to filter the text currently not in use

In [6]:

# def wordopt(t):
#     t = t.lower()
#     t = re.sub('\[.*?\]', '', t)
#     t = re.sub("\\W"," ",t)
#     t = re.sub('https?://\S+|www\.\S+', '', t)
#     t = re.sub('<.*?>+', '', t)
#     t = re.sub('[%s]' % re.escape(string.punctuation), '', t)
#     t = re.sub('\n', '', t)
#     t = re.sub('\w*\d\w*', '', t)
#     return t

In [7]:
dataframe=dataframe.dropna()

In [8]:
x=dataframe['text']
y=dataframe['label']
# x=x.apply(wordopt)
# tokenizer=Tokenizer()
# tokenizer.fit_on_texts(x)
# x=tokenizer.texts_to_sequences(x)
# x=pad_sequences(x,maxlen=1000,padding='post',truncating='post')

embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
x=embedding.embed_documents(x)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [21]:
len(x_train[0])

384

In [52]:
model=Sequential()
model.add(Dense(64,activation='relu',input_dim=384))
model.add(Dense(64,activation='relu'))



model.add(Dense(64,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))

model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [53]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [54]:
model.fit(np.array(x_train),np.array(y_train),epochs=10,batch_size=32,validation_data=(np.array(x_val),np.array(y_val)))

Epoch 1/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8048 - loss: 0.5604 - val_accuracy: 0.8897 - val_loss: 0.2740
Epoch 2/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8957 - loss: 0.2657 - val_accuracy: 0.8992 - val_loss: 0.2550
Epoch 3/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9128 - loss: 0.2289 - val_accuracy: 0.9059 - val_loss: 0.2407
Epoch 4/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9274 - loss: 0.1972 - val_accuracy: 0.9122 - val_loss: 0.2244
Epoch 5/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9355 - loss: 0.1783 - val_accuracy: 0.9115 - val_loss: 0.2212
Epoch 6/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9445 - loss: 0.1589 - val_accuracy: 0.9126 - val_loss: 0.2206
Epoch 7/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9487 - loss: 0.1466 - val_accuracy: 0.9115 - val_loss: 0.2278
Epoch 8/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9551 - loss: 0.1316 - 

In [55]:
y_predict=model.predict(np.array(x_test))
y_pred=np.where(y_predict>0.5,1,0)

411/411 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [56]:
classificationreport=classification_report(y_test,y_pred)

In [57]:
print(classificationreport)

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      6277
           1       0.91      0.93      0.92      6859

    accuracy                           0.92     13136
   macro avg       0.92      0.92      0.92     13136
weighted avg       0.92      0.92      0.92     13136



In [58]:
confusion_matrix(y_test,y_pred)

array([[5672,  605],
       [ 473, 6386]])

In [48]:
xgbclassifier=xgb.XGBClassifier(objective='binary:logistic',random_state=42)
xgbclassifier.fit(np.array(x_train),np.array(y_train))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [49]:
confusion_matrix(y_test,xgbclassifier.predict(x_test))

array([[5561,  716],
       [ 701, 6158]])

In [59]:
model.save('unreliable+news+detection+model.keras')

In [60]:
from google.colab import files
files.download('unreliable+news+detection+model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>